# Исследование надежности заемщиков.

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Цель проекта —** ответить на вопросы:
1. Есть ли зависимость между количеством детей и возвратом кредита в срок?
2. Есть ли зависимость между семейным положением и возвратом кредита в срок?
3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
4. Как разные цели кредита влияют на его возврат в срок?

**Ход исследования**
Данные о статистике платёжеспособности клиентов будут получены через путь:  `/datasets/data.csv`. О качестве данных ничего не известно, поэтому перед проверкой гипотез понадобиться обзор данных. 

Данные будут проверены на возможные ошибки. Затем, на этапе предобработки, мы попытаемся исправить самые критичные из них. 

Таким образом, исследование пройдет в три этапа:

1. Обзор данных.
2. Предобработка данных.
3. Ответы на вопросы.

### Шаг 1. Обзор данных

Составим первое представление о данных. Импортируем библиотеку `Pandas`:

In [1]:
import pandas as pd

теперь сохраним файл `data.csv` в переменную `data`. 

In [2]:
data = pd.read_csv('data.csv')

Выведем на экран первые 10 строк таблицы:

In [3]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Теперь получим общую информацию о таблице:

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


В таблице 11 столбцов. 

Согласно документации к данным:

* `children` — количество детей в семье
* `days_employed` — общий трудовой стаж в днях
* `dob_years` — возраст клиента в годах
* `education` — уровень образования клиента
* `education_id` — идентификатор уровня образования
* `family_status` — семейное положение
* `family_status_id` — идентификатор семейного положения
* `gender` — пол клиента
* `income_type` — тип занятости
* `debt` — имел ли задолженность по возврату кредитов
* `total_income` — ежемесячный доход
* `purpose` — цель получения кредита

Видны следующие проблемы:

1. В столбцах `days_employed` и `total_income` присутствуют пропуски.
2. Некоторые данные в столбце `days_employed` записаны, как отрицательное число.
3. В данных столбца `education` имеются неявные дубликаты в виде одних и тех же слов, записанных разными способами.
4. Данные в столбце `total_income` имеют вещественный тип, осложняя их обработку. 

**Выводы**

Предварительно можно утверждать, что предоставленных данных достаточно для исследования. Но сперва нужно устранить проблемы в данных.

### Шаг 2.1 Заполнение пропусков

Сперва заполним пропущенные значения в столбцах `days_employed` и `total_income`.

Видно, что в обоих столбцах число пропущенных значений одинаковое: 19351.

Определим, какую долю составляют пропущенные значения в каждом из столбцов с пропусками, поочередно применив методы `isna()` и `mean()`:

In [5]:
null_percentage = data.isna().mean()
print(null_percentage)

children            0.000000
days_employed       0.100999
dob_years           0.000000
education           0.000000
education_id        0.000000
family_status       0.000000
family_status_id    0.000000
gender              0.000000
income_type         0.000000
debt                0.000000
total_income        0.100999
purpose             0.000000
dtype: float64


Довольно высокий результат. Простое удаление пропусков может привести к сильным искажениям данных.

Самая вероятная причина пропусков в этих столбцах состоит в том, что эти клиенты попросту не имеют опыта работы. В результате чего ячейки с трудовым стажем и ежемесячным доходом не были заполнены. Альтернативно источником ошибки может быть человеческий фактор.

Заполним пропущенные ячейки столбца `total_income` медианным значением всех его строк:

In [6]:
total_income_median = data['total_income'].median()
data['total_income'] = data['total_income'].fillna(total_income_median)

Среднее значение некорректно характеризует данные, когда некоторые значения сильно выделяются среди большинства. Потому использование медианного значения в заполнении пустых ячеек с количественными данными - предпочтительнее.

### Шаг 2.2 Проверка данных на аномалии и исправления.

В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. Например, отрицательное количество дней трудового стажа в столбце `days_employed`. Для реальных данных это нормально.

Самая вероятная причина возникновения этой ошибки в человеческом факторе. По неизвестным причинам данные изначально были введены неверным образом.

Исправим ошибки в этом столбце, сначала сделав все его данные положительными числами:

In [7]:
data['days_employed'] = abs(data['days_employed'])

Теперь заполним пропуски медианными значениями, как в предыдущем шаге:

In [8]:
days_employed_median = data['days_employed'].median()
data['days_employed'] = data['days_employed'].fillna(days_employed_median)

Проверим остальные столбцы на присутствие скрытых аномалий.

Столбец `children`:

In [9]:
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Мы сразу же нашли несколько ошибок в данных. 

У некоторых клиентов есть "-1" ребенок. Также, целых 76 клиентов в списке имеют 20 детей. Учитывая, что количество таких многодетных клиентов многозначительно превышает предыдущее по размеру значение, маловероятно, что это верное значение.

Очевидно, что это - ошибки ввода. В первом случае имелось в виду "1", а во втором "2". 

Исправим эти значения, заменив их с помощью атрибута `loc`:

In [10]:
rows = (data['children'] == -1)
data.loc[rows, 'children'] = 1

rows = (data['children'] == 20)
data.loc[rows, 'children'] = 2

Проверим результат:

In [11]:
data['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

Столбец `days_employed`, использовав метод `describe()`:

In [12]:
data['dob_years'].describe()

count    21525.000000
mean        43.293380
std         12.574584
min          0.000000
25%         33.000000
50%         42.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64

Мы видим, что у некоторых клиентов в столбце возраста поставлен нуль. 

Иправим это, заменив нули на медианное значение остальных данных, и сразу же проверив резльтат:

In [13]:
rows = (data['dob_years'] == 0)
data.loc[rows, 'dob_years'] = data['dob_years'].median()

data['dob_years'].describe()

count    21525.000000
mean        43.490453
std         12.218595
min         19.000000
25%         34.000000
50%         42.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64

Столбец `education_id`:

In [14]:
data['education_id'].value_counts()

1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64

Аномалий нет.

Столбец `family_status_id`:

In [15]:
data['family_status_id'].value_counts()

0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64

Аномалий нет.

Столбец `gender`:

In [16]:
data['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

Обнаружено странное значение "XNA". Скорее всего кто-то решил оставить свой пол неопределенным. Поскольку это не критично для целей исследования, эту ячейку можно оставить, как есть.

Столбец `debt`:

In [17]:
data['debt'].value_counts()

0    19784
1     1741
Name: debt, dtype: int64

Аномалий нет.

Столбцы `days_employed` и `total_income` мы обработаем в следующем пункте.

### Шаг 2.3. Изменение типов данных.

Еще ранее мы заметили, что тип данных в столбце `days_employed` и `total_income` - вещественный.

Заменим вещественный тип данных в столбце `days_employed` на целочисленный и проверим изменения:

In [18]:
data['days_employed'] = data['days_employed'].astype('int')

data['days_employed'].dtype

dtype('int32')

Сделаем то же самое со столбцом `total_income`:

In [19]:
data['total_income'] = data['total_income'].astype('int')

data['total_income'].dtype

dtype('int32')

Проблема устранена.

### Шаг 2.4. Удаление дубликатов.

Проверим количество дубликатов в столбце `education`, сначала применив метод `sort_values()`для облегчения просмотра, а затем использовав метод `unique()`, чтобы вывести список с уникальными значениями:

In [20]:
data['education'].sort_values().unique()

array(['ВЫСШЕЕ', 'Высшее', 'НАЧАЛЬНОЕ', 'НЕОКОНЧЕННОЕ ВЫСШЕЕ',
       'Начальное', 'Неоконченное высшее', 'СРЕДНЕЕ', 'Среднее',
       'УЧЕНАЯ СТЕПЕНЬ', 'Ученая степень', 'высшее', 'начальное',
       'неоконченное высшее', 'среднее', 'ученая степень'], dtype=object)

Мы подтвердили, что все дупликаты возникли в результате использования нижнего и верхнего регистра.

Исправим это, применив метод `str.lower()` и проверив результат:

In [21]:
data['education'] = data['education'].str.lower()

data['education'].sort_values().unique()

array(['высшее', 'начальное', 'неоконченное высшее', 'среднее',
       'ученая степень'], dtype=object)

На всякий случай проверим остальные столбцы с вещественными данными, заранее исправив регистр.

Столбец `family_status`:

In [22]:
data['family_status'] = data['family_status'].str.lower()

data['family_status'].sort_values().unique()

array(['в разводе', 'вдовец / вдова', 'гражданский брак',
       'женат / замужем', 'не женат / не замужем'], dtype=object)

Дубликатов не обнаружено.

Столбец `income_type`:

In [23]:
data['income_type'] = data['income_type'].str.lower()

data['income_type'].sort_values().unique()

array(['безработный', 'в декрете', 'госслужащий', 'компаньон',
       'пенсионер', 'предприниматель', 'сотрудник', 'студент'],
      dtype=object)

Дубликатов не обнаружено.

Cтолбец `purpose`:

In [24]:
data['purpose'] = data['purpose'].str.lower()

data['purpose'].sort_values().unique()

array(['автомобили', 'автомобиль', 'высшее образование',
       'дополнительное образование', 'жилье',
       'заняться высшим образованием', 'заняться образованием',
       'на покупку автомобиля', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля', 'на проведение свадьбы',
       'недвижимость', 'образование', 'операции с жильем',
       'операции с коммерческой недвижимостью',
       'операции с недвижимостью', 'операции со своей недвижимостью',
       'покупка жилой недвижимости', 'покупка жилья',
       'покупка жилья для сдачи', 'покупка жилья для семьи',
       'покупка коммерческой недвижимости', 'покупка недвижимости',
       'покупка своего жилья', 'получение высшего образования',
       'получение дополнительного образования', 'получение образования',
       'приобретение автомобиля', 'профильное образование',
       'ремонт жилью', 'свадьба', 'свой автомобиль',
       'сделка с автомобилем', 'сделка с подержанным автомобилем',
       'строительство 

Здесь мы видим множество дубликатов. Заметна опечатка "ремонт жиль*ю*", но для исследования это не критично. Более подробно мы обработаем этот столбец позже.

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Для удобства работы с таблицей, мы можем перенести содержимое столбцов `education` и `family_status` в отдельные таблицы-словари, оставив в изначальном датафрейме только их идентификаторы.

Создадим датафрейм `education_dict`:

In [25]:
education_dict = data[['education','education_id']]
education_dict = education_dict.drop_duplicates().reset_index(drop=True)
display(education_dict)

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


Теперь создадим датафрейм `family_status_dict`:

In [26]:
family_status_dict = data[['family_status','family_status_id']]
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True)
display(family_status_dict)

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,не женат / не замужем,4


Осталось удалить `education` и `family_status` из исходного датафрейма и проверить результат: 

In [27]:
data.drop(columns=['education', 'family_status'])

data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


В дальнейшем, если нам понадобиться обратиться к этим данным, нам достаточно будет только просмотреть их идентификатор.

### Шаг 2.6. Категоризация дохода.

Добавим в исходную таблицу новый столбец, который будет содержать категории клиентов на основе информации об их доходе. 

Для этого создадим функцию `total_income_category`. На вход она должна принимать значение столбца `total_income`, а на выход выдавать название категории:

In [28]:
def total_income_category(total_income):
    if total_income <= 30000:
        return 'E'
    if total_income <= 50000:
        return 'D'
    if total_income <= 200000:
        return 'C'
    if total_income <= 1000000:
        return 'B'
    return 'A'

Вызовем функцию `total_income_category()` для столбца `total_income` и сохраним результат в новый столбец `total_income_category`:

In [29]:
data['total_income_category'] = data['total_income'].apply(total_income_category)

Проверим результат, выведя первые десять строк столбцов `total_income`и `total_income_category`:

In [30]:
data.loc[:10, ['total_income','total_income_category']]

,total_income,total_income_category
0,253875,B
1,112080,C
2,145885,C
3,267628,B
4,158616,C
5,255763,B
6,240525,B
7,135823,C
8,95856,C
9,144425,C


Все сходится.

### Шаг 2.7. Категоризация целей кредита.

Значения в столбце `purpose` можно объединить по категориям:

* **проведение свадьбы:** свадьба, на проведение свадьбы, сыграть свадьбу
* **операции с недвижимостью:** недвижимость, операции с недвижимостью, покупка коммерческой недвижимости, операции с жильем, покупка жилья для сдачи, операции с коммерческой недвижимостью, жилье, покупка жилья, покупка жилья для семьи, строительство собственной недвижимости, операции со своей недвижимостью, строительство жилой недвижимости, покупка недвижимости, покупка своего жилья, строительство недвижимости, ремонт жилью, покупка жилой недвижимости.
* **операции с автомобилем:** автомобиль, на покупку своего автомобиля, сделка с подержанным автомобилем, свой автомобиль, на покупку подержанного автомобиля, автомобили, на покупку автомобиля, приобретение автомобиля, сделка с автомобилем.
* **получение образования:** образование, заняться высшим образованием, дополнительное образование, высшее образование, получение дополнительного образования, получение образования, профильное образование, получение высшего образования, заняться образованием.

Сформируем новый столбец, в котором будут находиться эти категории.

Для этого, создадим функцию `purpose_category()`. На вход она должна принимать значение столбца `purpose`, а на выход выдавать название категории цели.

In [31]:
def purpose_category(purpose):
    wedding_category_list = ['свадьба', 'на проведение свадьбы', 'сыграть свадьбу']
    housing_category_list = ['недвижимость', 'операции с недвижимостью', 'покупка коммерческой недвижимости', 
                             'операции с жильем', 'покупка жилья для сдачи', 'операции с коммерческой недвижимостью', 
                             'жилье', 'покупка жилья', 'покупка жилья для семьи', 'строительство собственной недвижимости', 
                             'операции со своей недвижимостью', 'строительство жилой недвижимости', 'покупка недвижимости', 
                             'покупка своего жилья', 'строительство недвижимости', 'ремонт жилью', 'покупка жилой недвижимости']  
    car_category_list = ['автомобиль', 'на покупку своего автомобиля', 'сделка с подержанным автомобилем', 
                         'свой автомобиль', 'на покупку подержанного автомобиля', 'автомобили', 
                         'на покупку автомобиля', 'приобретение автомобиля', 'сделка с автомобилем']
    education_category_list = ['образование', 'заняться высшим образованием', 'дополнительное образование', 
                               'высшее образование', 'получение дополнительного образования', 'получение образования', 
                               'профильное образование', 'получение высшего образования', 'заняться образованием']
    if purpose in wedding_category_list:
        return 'проведение свадьбы'
    if purpose in housing_category_list:
        return 'операции с недвижимостью'
    if purpose in car_category_list:
        return 'операции с автомобилем'
    if purpose in education_category_list:
        return 'получение образования'
    return 'неизвестно'

Альтернативный вариант функции `purpose_category()`:

In [32]:
def purpose_category(purpose):
    if 'свадьб' in purpose:
        return 'проведение свадьбы'
    if 'недвиж' in purpose:
        return 'операции с недвижимостью'
    if 'жил' in purpose:
        return 'операции с недвижимостью'
    if 'автомоб' in purpose:
        return 'операции с автомобилем'
    if 'образов' in purpose:
        return 'получение образования'
    return 'неизвестно'

Вызовем функцию `purpose_category()` для столбца `purpose` и сохраним результат в новый столбец `purpose_category`:

In [33]:
data['purpose_category'] = data['purpose'].apply(purpose_category)

Проверим результат, выведя первые десять строк столбца `purpose` и `purpose_category`:

In [34]:
data.loc[:10, ['purpose','purpose_category']]

,purpose,purpose_category
0,покупка жилья,операции с недвижимостью
1,приобретение автомобиля,операции с автомобилем
2,покупка жилья,операции с недвижимостью
3,дополнительное образование,получение образования
4,сыграть свадьбу,проведение свадьбы
5,покупка жилья,операции с недвижимостью
6,операции с жильем,операции с недвижимостью
7,образование,получение образования
8,на проведение свадьбы,проведение свадьбы
9,покупка жилья для семьи,операции с недвижимостью


Все сходится.

### Ответы на вопросы.

Теперь мы готовы ответить на вопросы, поставленные в начале исследования. 

Чтобы найти на них, для начала узнаем средний процент клиентов, которые возвращают долг вовремя.

In [35]:
x = data[data['debt'] == 0]['debt'].count()
y = data['debt'].count()
return_percentage = x / y
print(f'Процент возврата: {return_percentage:.0%}')

Процент возврата: 92%


Мы будем использовать это соотношение, как показатель для определения зависимости между различными данными и способностью клиента возвращать долг вовремя.

##### Вопрос 1:

**Есть ли зависимость между количеством детей и возвратом кредита в срок?**

Теперь мы составим сводную таблицу. В качестве строк мы возьмем уникальные значения столбца `children`. Стобец `0` - обозначает количество клиентов без задолженности. Столбец `1` - количество клиентов с задолженностью. Мы также добавим столбец с соотношением значений столбца `0` к сумме столбцов, чтобы сравнивать его с процентом, полученным ранее.

In [36]:
family_pivot = data.pivot_table(index=['children'], columns='debt', values='total_income', aggfunc='count')
family_pivot['ratio'] = family_pivot[0] / (family_pivot[0] + family_pivot[1])
display(family_pivot)

debt,0,1,ratio
children,,,
0,13086.0,1063.0,0.924871
1,4420.0,445.0,0.908530
2,1929.0,202.0,0.905209
3,303.0,27.0,0.918182
4,37.0,4.0,0.902439
5,9.0,NaN,NaN


##### Вывод 1:

Мы можем наблюдать, что способность возвращать долг выше у клиентов без детей. Дальше наблюдается снижение показателя - чем больше детей, тем меньше вероятность вернуть кредит. Определить показатель для клиентов с 5 детьми пока не представляется возможным в виду их маленького числа.

##### Вопрос 2:

**Есть ли зависимость между семейным положением и возвратом кредита в срок?**

Для ответа на этот вопрос мы снова составим сводную таблицу. На этот раз в качестве строк мы будем использовать уникальные значения столбца `family_status_id`.

In [37]:
family_pivot = data.pivot_table(index=['family_status_id'], columns='debt', values='total_income', aggfunc='count')
family_pivot['ratio'] = family_pivot[0] / (family_pivot[0] + family_pivot[1])
display(family_pivot)

debt,0,1,ratio
family_status_id,,,
0,11449,931,0.924798
1,3789,388,0.907110
2,897,63,0.934375
3,1110,85,0.928870
4,2539,274,0.902595


Чтобы было проще определить, что обозначает номер категории семейного статуса, используем метод `merge()`, чтобы соединить сводную таблицу с соответствующей таблицей-словарем:.

In [38]:
family_pivot = family_pivot.merge(family_status_dict, on='family_status_id', how='left')

display(family_pivot)

,family_status_id,0,1,ratio,family_status
0,0,11449,931,0.924798,женат / замужем
1,1,3789,388,0.907110,гражданский брак
2,2,897,63,0.934375,вдовец / вдова
3,3,1110,85,0.928870,в разводе
4,4,2539,274,0.902595,не женат / не замужем


##### Вывод 2:

Мы наблюдаем повышенный показатель для всех вариантов, кроме клиентов, находящихся в гражданском браке и тех, кто не женат / не замужем. 

#####  Вопрос 3:

**Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

Снова составим сводную таблицу. Строками теперь послужат уникальные значения столбца `total_income_category`:

In [39]:
income_pivot = data.pivot_table(index=['total_income_category'], columns='debt', values='total_income', aggfunc='count')
income_pivot['ratio'] = income_pivot[0] / (income_pivot[0] + income_pivot[1])
display(income_pivot)

debt,0,1,ratio
total_income_category,,,
A,23,2,0.920000
B,4685,356,0.929379
C,14727,1360,0.915460
D,329,21,0.940000
E,20,2,0.909091


#####  Вывод 3:

Мы наблюдаем слегка повышенную возможность возможность кредит в срок для клиентов категории B и значительно более высокую возможность для категории D. Возможность ниже нормы для категории E.

#####  Вопрос 4:

**Как разные цели кредита влияют на его возврат в срок?**

В последний раз составим сводную таблицу. Строками послужат уникальные значения столбца `purpose_category`:

In [40]:
purpose_pivot = data.pivot_table(index=['purpose_category'], columns='debt', values='total_income', aggfunc='count')
purpose_pivot['ratio'] = purpose_pivot[0] / (purpose_pivot[0] + purpose_pivot[1])
display(purpose_pivot)

debt,0,1,ratio
purpose_category,,,
операции с автомобилем,3912,403,0.906605
операции с недвижимостью,10058,782,0.927860
получение образования,3652,370,0.908006
проведение свадьбы,2162,186,0.920784


#####  Вывод 4:

Мы наблюдаем, что способность возвращать кредит в срок падает, если причиной взятия кредита были операции с автомобилем или получение образования.

## Общий вывод:

Мы ответили на четыре вопроса и постановили:

1. Количество детей влияет на способность оплачивать кредит в срок.
2. Семейное положение влияет на способность оплачивать кредит в срок.
3. Уровень дохода влияет на способность оплачивать кредит в срок.
4. Некоторые цели кредита негативно влияют на возможность возврата кредита в срок, а именно "операции с автомобилем" и "получение образования". 

Несмотря на разницу лишь в паре процентов, способность клиента оплачивать кредит в срок меняется от большого множества факторов, каждый из которых стоит учитывать при построении модели кредитного скоринга.